In [1]:
import pandas as pd
import numpy as np

In [2]:
epigenetics = pd.read_excel("Epigenetic age_Cleaned_18042024.xlsx")
microbiome = pd.read_excel("src/summary_report/6_taxonomy_community/6_Genus/Genus_abund_all.xlsx")
SG90 = pd.read_excel("SG90_TimWehnes_26082024.xlsx")
BMI = pd.read_csv("BMI.csv")
#selected columns of epigenetic data
epigenetic_processed = pd.read_csv("Epigenetic_Processed.csv")
genus_clr = pd.read_csv("Genus_CLR.csv")
genus_raw = pd.read_csv("Genus_Raw.csv")

In [3]:
epigenetic_ids = epigenetics["Samples"].astype(str)
microbiome_ids = pd.Series(microbiome.columns[1:].str.replace(r'^[A-Za-z]_', '', regex=True))
# Find the intersection of df ids and the combined set
overlapping_samples = set(epigenetic_ids).intersection(microbiome_ids)

In [137]:
#transpose and format microbiome csv for compatibility
m2 = microbiome.transpose()
m2.reset_index
m2.columns = m2.loc["Genus"]
m2 = m2.iloc[1:]
m2['Sample_ID'] = m2.index  # Move the current index to a new column 
m2 = m2.reset_index(drop=True)  # Reset the index to be numeric starting from 0
m2.index.name = None 
# Reorder columns to have samples as the first column
m2 = m2[['Sample_ID'] + [col for col in m2.columns if col != 'Sample_ID']]
m2["Sample_ID"] = microbiome_ids
filtered_m2 = m2[m2['Sample_ID'].isin(overlapping_samples)]
filtered_m2 = filtered_m2.reset_index(drop=True)
filtered_m2.columns.name = None
#final df
df = filtered_m2

In [138]:
# Convert Sample_ID to string in both DataFrames
df['Sample_ID'] = df['Sample_ID'].astype(str)
epigenetic_processed['Sample_ID'] = epigenetic_processed['Sample_ID'].astype(str)

#merge on overlapping samples, sample 80004 has had blood taken twice
df = pd.merge(df, epigenetic_processed, on='Sample_ID', suffixes=('', '_epigenetic'))
#df

In [139]:
# Correct formatting of alternative IDs
SG90.loc[SG90["alt_ID"].notna(), "alt_ID"] = SG90.loc[SG90["alt_ID"].notna(), "alt_ID"].astype(int)

# Convert IDs to string
ids = df["Sample_ID"].astype(str)
subno_ids = SG90["subno"].astype(str)

# Combine subno and alt_ID into one set of possible IDs
alt_ids = SG90["alt_ID"].dropna().astype(int).astype(str)  # Drop NaN values and convert to string

# Combine both subno_ids and alt_ids into a single set
combined_ids_SG90 = set(subno_ids).union(set(alt_ids))

# Find the intersection of df ids and the combined set
overlapping_samples2 = set(ids).intersection(combined_ids_SG90)

# Count the number of overlapping IDs
num_overlapping_samples2 = len(overlapping_samples2)
print(f"Number of overlapping sample IDs: {num_overlapping_samples2}")

Number of overlapping sample IDs: 317


In [140]:
# Convert Sample_ID, subno, and alt_ID to strings for comparison
ids = df["Sample_ID"].astype(str)
SG90["subno"] = SG90["subno"].astype(str)
SG90["alt_ID"] = SG90["alt_ID"].dropna().astype(int).astype(str)

# Identify rows in SG90 where either subno or alt_ID matches Sample_ID in df
matching_rows = SG90[(SG90["subno"].isin(ids)) | (SG90["alt_ID"].isin(ids))].copy()

# Create a new 'Sample_ID' column in result_df
matching_rows["Sample_ID"] = matching_rows.apply(
    lambda row: row["subno"] if row["subno"] in ids.values else row["alt_ID"],
    axis=1
)

# Select the first 20 columns
first_20_columns = matching_rows.iloc[:, :20]

# Select columns starting with "demogr"
demogr_columns = matching_rows.filter(regex='^demogr')

# Select specific columns by name
specific_columns = matching_rows[
    [
         
        "ansur_frax_bmi_crit", 
        #"med_cond_frax_height",
        #"med_cond_frax_weight",
        "ansur_frax_bmi",
        "subs_use_smoke_consolidated", 
        "subs_use_smoke_freq_consolidated", 
        "subs_use_last_smoke_consolidated", 
        "subs_use_beer_consolidated", 
        "subs_use_rice_wine_consolidated", 
        "subs_use_grape_wine_consolidated", 
        "subs_use_hard_liquor_consolidated", 
        "others_v1_date", 
        "others_v4_date", 
        "others_v5_date", 
        "others_Blood_collection_date", 
        "others_Saliva_collection_date",
        
    ]
]

# Concatenate all the selected columns
result_df = pd.concat([matching_rows[["Sample_ID"]], first_20_columns, demogr_columns, specific_columns], axis=1)

# Display the resulting DataFrame
#result_df

In [141]:
# Merge into final df
merged_df = pd.merge(df, result_df, on="Sample_ID", how="left")

# Display the merged DataFrame
#merged_df

In [142]:
# Identify duplicate Sample_ID values in merged_df
duplicate_sample_ids = merged_df[merged_df.duplicated('Sample_ID', keep=False)]

# Display the rows with duplicate Sample_ID values
#duplicate_sample_ids


In [143]:
#rows 0,1,3,7 removed bc of likely wrong collection date - not possible that it has been collected twice since measurements are identical
merged_df = merged_df.drop([0, 1, 3, 7])
merged_df = merged_df.reset_index(drop=True)

# Check if blood and saliva have equal dates
# Identify rows where the dates are not equal in merged_df
unequal_dates_rows = merged_df[merged_df['others_Blood_collection_date'] != merged_df['others_Saliva_collection_date']]

# remove the 7rows where the dates are not equal
merged_df = merged_df.drop(unequal_dates_rows.index)
merged_df = merged_df.reset_index(drop=True)

In [144]:
# Drop the specified columns from merged_df
for column in [
    'subno', 
    'redcap_event_name', 
    'f4_no', 
    'alt_ID', 
    'subno_schs', 
    'Date_of_blood_collection', 
    'group', 
    'status', 
    'participant_study_status_complete'
]:
    if column in merged_df.columns:
        merged_df = merged_df.drop(columns=column)

In [145]:
# Identify columns with the same name (duplicates)
duplicate_columns = merged_df.columns[merged_df.columns.duplicated()].tolist()

# Display the list of duplicate columns
print(f"Duplicate column names: {duplicate_columns}")
# Drop the duplicates by keeping only the first occurrence
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]


Duplicate column names: ['demogr_doi', 'demogr_dob', 'demogr_age', 'demogr_sex', 'demogr_race', 'demogr_edu_level', 'demogr_sch_year', 'demogr_house_type', 'demogr_cur_employ']


In [146]:
merged_df.iloc[0:3,1:545]

,g__Prevotella_7,g__Streptococcus,g__Neisseria,g__Veillonella,g__Haemophilus,g__Prevotella,g__Alloprevotella,g__Rothia,g__Porphyromonas,g__Fusobacterium,...,g__Pseudoflavonifractor,g__Catabacter,g__Eubacterium]_oxidoreducens_group,g__SM2D12_unclassified,g__Paraeggerthella,g__Macrococcus,g__Subgroup_7_unclassified,g__Brevibacterium,g__Luteitalea,g__Negativicoccus
0,25.422162,2.927833,5.599429,9.283374,3.969587,6.699992,1.529026,0.47467,1.789465,3.238679,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,29.107075,18.73805,1.307839,10.110899,3.185468,2.611855,8.017208,0.751434,0.357553,1.869981,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,33.242217,12.063594,1.059233,19.383756,1.173799,2.102386,2.25514,1.589854,0.233152,0.410026,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [147]:
merged_df

,Sample_ID,g__Prevotella_7,g__Streptococcus,g__Neisseria,g__Veillonella,g__Haemophilus,g__Prevotella,g__Alloprevotella,g__Rothia,g__Porphyromonas,...,subs_use_last_smoke_consolidated,subs_use_beer_consolidated,subs_use_rice_wine_consolidated,subs_use_grape_wine_consolidated,subs_use_hard_liquor_consolidated,others_v1_date,others_v4_date,others_v5_date,others_Blood_collection_date,others_Saliva_collection_date
0,80004,25.422162,2.927833,5.599429,9.283374,3.969587,6.699992,1.529026,0.47467,1.789465,...,NaN,1.0,1.0,1.0,1.0,2018-02-28 00:00:00,2018-03-21 00:00:00,2019-03-14 00:00:00,2021-01-14 00:00:00,2021-01-14 00:00:00
1,80001,29.107075,18.73805,1.307839,10.110899,3.185468,2.611855,8.017208,0.751434,0.357553,...,NaN,1.0,NaN,1.0,1.0,#NULL!,#NULL!,#NULL!,2015-11-30 00:00:00,2015-11-30 00:00:00
2,80020,33.242217,12.063594,1.059233,19.383756,1.173799,2.102386,2.25514,1.589854,0.233152,...,NaN,1.0,NaN,1.0,1.0,#NULL!,#NULL!,#NULL!,2015-12-10 00:00:00,2015-12-10 00:00:00
3,80007,11.693168,19.713437,20.926088,4.543072,10.567884,3.120741,1.041412,1.181199,1.91508,...,NaN,3.0,1.0,1.0,1.0,2017-10-20 00:00:00,2017-10-23 00:00:00,2018-03-16 00:00:00,2018-09-20 00:00:00,2018-09-20 00:00:00
4,80019,23.716786,16.067521,0.171929,18.09628,2.791497,2.300719,1.190997,8.877774,0.012504,...,NaN,1.0,NaN,1.0,1.0,#NULL!,#NULL!,#NULL!,2015-12-15 00:00:00,2015-12-15 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,43327,33.492615,26.418022,0.0,22.433908,0.0,0.0,0.0,7.254561,0.031029,...,NaN,1.0,1.0,1.0,1.0,2018-03-15 00:00:00,2018-03-19 00:00:00,2018-11-21 00:00:00,2019-10-02 00:00:00,2019-10-02 00:00:00
307,47170,1.947246,5.377852,32.617725,2.626895,28.437348,4.423108,6.203139,2.750957,1.990399,...,7.0,1.0,1.0,1.0,1.0,2018-01-25 00:00:00,2018-01-26 00:00:00,2018-11-02 00:00:00,2019-10-02 00:00:00,2019-10-02 00:00:00
308,5703,14.917127,6.385109,22.596314,10.423078,7.697727,3.040528,4.571916,2.183989,3.667173,...,NaN,1.0,1.0,1.0,1.0,2018-03-08 00:00:00,2018-03-23 00:00:00,2018-09-24 00:00:00,2019-10-08 00:00:00,2019-10-08 00:00:00
309,14654,49.30186,14.795567,0.125227,7.225596,2.886482,0.713794,4.019786,2.560892,2.492017,...,7.0,1.0,1.0,1.0,1.0,2018-03-22 00:00:00,2018-03-23 00:00:00,2018-09-18 00:00:00,2019-10-09 00:00:00,2019-10-09 00:00:00


In [148]:
# Now, remove cols 
columns_to_remove = [
    'demogr_intv_pl', 
    'subs_use_smoke_freq_consolidated', 
    'subs_use_last_smoke_consolidated', 
    'demogr_sch_year', 
    'demogr_dob',
    "demogr_edu_level",
    "demogr_intv_met",
    "demogr_edu_level",
    "demogr_cur_employ",
    "demogr_house_type",

]

# Removing columns that start with "soc_"
columns_to_remove += [col for col in merged_df.columns if col.startswith('soc_')]

# Dropping the specified columns
merged_df_cleaned = merged_df.drop(columns=columns_to_remove)
df = merged_df_cleaned

In [149]:
# Replace '#NULL!' with NaN
df.replace('#NULL!', np.nan, inplace=True)

# Convert date columns to datetime
df['others_v1_date'] = pd.to_datetime(df['others_v1_date'], errors='coerce')
df['others_Blood_collection_date'] = pd.to_datetime(df['others_Blood_collection_date'], errors='coerce')
df['others_v4_date'] = pd.to_datetime(df['others_v4_date'], errors='coerce')
df['others_v5_date'] = pd.to_datetime(df['others_v5_date'], errors='coerce')

# Calculate time differences and handle NaN values
df['time_diff_demogr_blood'] = df.apply(
    lambda row: np.abs((row['others_v1_date'] - row['others_Blood_collection_date']).days) 
    if pd.notna(row['others_v1_date']) and pd.notna(row['others_Blood_collection_date']) else np.nan, 
    axis=1
)

df['time_diff_subst_blood'] = df.apply(
    lambda row: np.abs((row['others_v4_date'] - row['others_Blood_collection_date']).days) 
    if pd.notna(row['others_v4_date']) and pd.notna(row['others_Blood_collection_date']) else np.nan, 
    axis=1
)

df['time_diff_bmi_blood'] = df.apply(
    lambda row: np.abs((row['others_v5_date'] - row['others_Blood_collection_date']).days) 
    if pd.notna(row['others_v5_date']) and pd.notna(row['others_Blood_collection_date']) else np.nan, 
    axis=1
)


C:\Users\wehne\AppData\Local\Temp\ipykernel_24280\488929314.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['time_diff_demogr_blood'] = df.apply(
C:\Users\wehne\AppData\Local\Temp\ipykernel_24280\488929314.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['time_diff_subst_blood'] = df.apply(
C:\Users\wehne\AppData\Local\Temp\ipykernel_24280\488929314.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join

In [150]:
#Interaction Test conducted to check significance of time difference from (ind)dependent variables to confounders
#since not significant:
# List of specific columns to remove
columns_to_remove = ['time_diff_demogr_blood', 'time_diff_subst_blood', 'time_diff_bmi_blood', 
                     'others_v1_date', 'others_v4_date', 'others_v5_date', 
                     'others_Saliva_collection_date', 'others_Blood_collection_date', 
                     'demogr_age', 'ansur_frax_bmi_crit', "demogr_doi"]

# Adding columns that start with 'soc_' to the list of columns to remove
columns_to_remove += [col for col in df.columns if col.startswith('soc_')]

# Dropping the specified columns
df_cleaned = df.drop(columns=columns_to_remove)

# Updating df to the cleaned DataFrame
df = df_cleaned


In [151]:
df.to_csv("Final_df.csv", index=False)

In [153]:
df

,Sample_ID,g__Prevotella_7,g__Streptococcus,g__Neisseria,g__Veillonella,g__Haemophilus,g__Prevotella,g__Alloprevotella,g__Rothia,g__Porphyromonas,...,Epigenetic_average,Epigenetic_deviation,demogr_sex,demogr_race,ansur_frax_bmi,subs_use_smoke_consolidated,subs_use_beer_consolidated,subs_use_rice_wine_consolidated,subs_use_grape_wine_consolidated,subs_use_hard_liquor_consolidated
0,80004,25.422162,2.927833,5.599429,9.283374,3.969587,6.699992,1.529026,0.474670,1.789465,...,89.544023,1.582379,2.0,1.0,17.545506,0.0,1.0,1.0,1.0,1.0
1,80001,29.107075,18.738050,1.307839,10.110899,3.185468,2.611855,8.017208,0.751434,0.357553,...,70.553625,-17.008019,2.0,1.0,23.871528,0.0,1.0,NaN,1.0,1.0
2,80020,33.242217,12.063594,1.059233,19.383756,1.173799,2.102386,2.255140,1.589854,0.233152,...,81.538284,-7.461716,2.0,1.0,25.778835,1.0,1.0,NaN,1.0,1.0
3,80007,11.693168,19.713437,20.926088,4.543072,10.567884,3.120741,1.041412,1.181199,1.915080,...,73.670066,-12.790208,1.0,1.0,27.038133,0.0,3.0,1.0,1.0,1.0
4,80019,23.716786,16.067521,0.171929,18.096280,2.791497,2.300719,1.190997,8.877774,0.012504,...,81.852874,-7.160825,2.0,1.0,9999.000000,0.0,1.0,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,43327,33.492615,26.418022,0.000000,22.433908,0.000000,0.000000,0.000000,7.254561,0.031029,...,80.429665,-6.386774,2.0,1.0,29.299593,0.0,1.0,1.0,1.0,1.0
307,47170,1.947246,5.377852,32.617725,2.626895,28.437348,4.423108,6.203139,2.750957,1.990399,...,80.676994,-6.246294,1.0,1.0,20.115458,2.0,1.0,1.0,1.0,1.0
308,5703,14.917127,6.385109,22.596314,10.423078,7.697727,3.040528,4.571916,2.183989,3.667173,...,77.016686,-12.695643,2.0,1.0,19.223376,0.0,1.0,1.0,1.0,1.0
309,14654,49.301860,14.795567,0.125227,7.225596,2.886482,0.713794,4.019786,2.560892,2.492017,...,77.126057,-12.169833,1.0,1.0,29.927159,2.0,1.0,1.0,1.0,1.0
